In [ ]:
import time
import calc
import numpy as np
import pandas as pd
import copy
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib import cm
from matplotlib import rcParams as rcParams
rcParams['figure.dpi'] = 300
%matplotlib inline
plt.style.use('dark_background')

# Basic investigation structure
nruns = 100
median_Hts = False        # If true, calculates mantle HPE budgets using median estimates in crust and BSE.
startpoint = 'latearc'   # starting time snapshot in model; present, phanero, latearc, midarc
endpoint = 'midarc'      # ending time snapshot in model; present, phanero, latearc, midarc
vary = 'Everything'         # Parameters to vary according to statistical distributions, or via gridding; see 'scenarios'
curves = ['HalfBy2.0']      # [i for i in calc.curvenames] # Crustal growth curve to assume. You may choose one or more from:
                         # ['HalfBy2.5', 'Linear', 'Sqrt', 'Fifth', 'HalfBy2.0', 'HalfBy2.5', 'HalfBy3.0', 'HalfBy3.5']
targetP = 3.6            # Reference pressure for thermochemical parameters. Recommendation: >3.36 GPa.

# Transition parameters from stagnant-lid to plate tectonic regime
duration=0.5
SL_at=3.686
SL_b=0.2
PT_at=2.84
PT_b=0.3


timestep=10.0 ** (-1 * calc.decimals)
tmin, tmax = calc.tmin, calc.tmax
assign_colors = cm.get_cmap('rainbow', len(calc.curvenames))
my_cm = dict(zip(calc.curvenames, np.linspace(0, 1, len(calc.curvenames))))
best_alpha = calc.cloud_alpha(nruns)
scenarios = {'Only_Start_End_Points': ['Ea_list', 'Qm_list', 'Tp_present'], 
    'Present_T_Flux' : ['Ea_list', 't_midarc', 'Tp_midarc', 't_latearc', 'Tp_latearc', 't_phanero', 'Tp_phanero'],
    'Rheology': ['t_midarc', 'Tp_midarc', 't_latearc', 'Tp_latearc', 't_phanero', 'Tp_phanero'],
    'TectonicTiming': ['Ea_list', 't_midarc', 'Tp_midarc', 't_latearc', 'Tp_latearc', 't_phanero', 'Tp_phanero'],
    'Everything': [],
       'Nothing': ['Ea_list', 'Qm_list', 'Tp_present', 't_midarc', 'Tp_midarc', 't_latearc', 'Tp_latearc', 't_phanero', 'Tp_phanero']
       }
constants = scenarios[vary]


In [ ]:
Isos, bse_budgets, cr_budgets, bse_Ht_max, crust_Ht_max = calc.HPE_budgets(nruns=nruns, timestep=timestep, trange=[tmin, tmax])
if median_Hts==True:
    Isos, bse_budgets, cr_budgets, bse_Ht_max, crust_Ht_max = calc.HPE_budgets(nruns=nruns*10, timestep=timestep, trange=[tmin, tmax])
    bse_Ht_median = copy.deepcopy(calc.CI_rows(bse_Ht_max)['50%'])
    crust_Ht_median = copy.deepcopy(calc.CI_rows(crust_Ht_max)['50%'])
    bse_Ht_max, crust_Ht_max = pd.DataFrame(index=bse_Ht_max.index), pd.DataFrame(index=crust_Ht_max.index)
    for i in range(nruns):
        bse_Ht_max[i] = bse_Ht_median
        crust_Ht_max[i] = crust_Ht_median
cases = calc.case_defs(nruns=nruns, constants=constants)
beta_budgets = calc.beta_budgets(nruns=nruns, timestep=timestep, SL_at=SL_at, SL_b=SL_b, PT_at=PT_at, PT_b=PT_b, duration=duration)
GrowthCurves = calc.growth_models(timestep=timestep).loc[list(crust_Ht_max.index), :]
UM, LM, WM = calc.PTXgrids()
Pslice = calc.slice_PTXgrid(grid=UM, targetP=targetP)
pz = pd.read_csv('PREM.csv', header=0, usecols=[8,9])
z_fP = interpolate.interp1d(pz['Pressure(GPa)'], pz['Depth(km)'])
dT_fTp = 0.0 #0.35 * z_fP(targetP)
rho_fT = interpolate.interp1d(Pslice['Tp'], Pslice['rho'])
alpha_fT = interpolate.interp1d(Pslice['Tp'], Pslice['alpha'])
Cp_fT = interpolate.interp1d(Pslice['Tp'], Pslice['Cp'])
cases['rho_ref'] = rho_fT(cases['Tp_present']+dT_fTp)
cases['Cp_ref'] = Cp_fT(cases['Tp_present']+dT_fTp)
cases['alpha_ref'] = alpha_fT(cases['Tp_present']+dT_fTp)
cases['deltaT_ref'] = cases['Tp_present']-dT_fTp-300.0

if cases['t_'+startpoint].mean()>cases['t_'+endpoint].mean():
    direction = -1  # running model FORWARD in time
    starttdiff = []
    endtdiff = []
    for i in range(len(cases.index)):
        starttdiff.append(cases['t_'+startpoint].iloc[i] - calc.round_down(cases['t_'+startpoint].iloc[i]))
        endtdiff.append(calc.round_up(cases['t_'+endpoint].iloc[i]) - cases['t_'+endpoint].iloc[i])
else:
    direction = 1  # running model BACKWARD in time
    starttdiff = []
    endtdiff = []
    for i in range(len(cases.index)):
        starttdiff.append(calc.round_up(cases['t_'+startpoint].iloc[i]) - cases['t_'+startpoint].iloc[i])
        endtdiff.append(cases['t_'+endpoint].iloc[i] - calc.round_down(cases['t_'+endpoint].iloc[i]))

cases['dBdt'] = ((SL_b - PT_b)/(cases['t_midarc'] - cases['t_latearc']))
cases['b_ints'] = PT_b - cases['t_latearc'] * cases['dBdt']

cases['endtdiff'] = endtdiff
cases['starttdiff'] = starttdiff
runs = cases.transpose()

calc.CI_cols(cases[list(i for i in cases if i!='Ea_list')])

In [ ]:
isodf=bse_budgets.transpose()
assign_colors2 = cm.get_cmap('rainbow', len(Isos))
my_cm2 = dict(zip(Isos.index, np.linspace(0, 1, len(Isos))))
hts=[]
ts=[]
for iso in isodf:
    del hts
    del ts
    ts=[]
    hts=[]
    summary = calc.CI_cols(isodf[iso])
    for t in bse_Ht_max.index:
        hts.append((summary*np.exp(Isos.loc[iso]['Lambda']*t)))
        ts.append(len((summary*np.exp(Isos.loc[iso]['Lambda']*t)))*[t])
    #tempdf = pd.DataFrame({'ts': ts, 'hts': hts}, index=range(len(ts)))
    #plt.plot(tempdf['ts'], tempdf['hts'], c=assign_colors(my_cm[iso]), alpha=0.1, label=iso)
    plt.plot(ts, hts, c=assign_colors2(my_cm2[iso]))

bse_statsum = calc.CI_rows(bse_Ht_max)
for i in range(len(bse_statsum.columns)):
    ialpha = 1- np.abs(i-2)/3 # 1/np.abs(np.log(np.abs(i-3)))
    plt.plot(bse_Ht_max.index, bse_statsum[bse_statsum.columns[i]], rasterized=True, c='orange', alpha=ialpha, lw=3*ialpha)

for iso in isodf:
    plt.plot(ts[0], hts[0], c=assign_colors2(my_cm2[iso]), label=iso, rasterized=True)

#plt.plot(beta_budgets.index, (beta_budgets.values * 100.)[:,int(0.5*len(beta_budgets.columns))], c='w', rasterized=True, label='Median '+r'$\beta$'+'$\cdot$'+'100')
plt.legend()
plt.show()

for curve in GrowthCurves:
    # Apply crustal growth model to bulk silicate earth radionuclide budget.
    mantle_Ht_max = calc.impose_growth_models_on_mantle(models=[curve],
        GrowthCurves=GrowthCurves, bse_df=bse_Ht_max, crust_df=crust_Ht_max)
    summary = calc.CI_rows(mantle_Ht_max)
    plt.plot(summary, c=assign_colors(my_cm[curve])) #, alpha=0.2)
plt.title('Uncertainty in Heat Production')
plt.show()

In [ ]:
outputs = pd.DataFrame(index=crust_Ht_max.index)
outputs2 = pd.DataFrame(index=crust_Ht_max.index)

for curve in curves:
    # Apply crustal growth model to bulk silicate earth radionuclide budget.
    mantle_Ht_max = calc.impose_growth_models_on_mantle(models=[curve], GrowthCurves=GrowthCurves, bse_df=bse_Ht_max, crust_df=crust_Ht_max)
    Tlist = len(runs.columns)*['NaN']  # Get ready to receive final mantle temperatures.
    
    for r in runs:  # For each case to be explored...
        Tp = runs[r]['Tp_'+startpoint]  # Pick an empirically-derived starting temperature...
        t_start = runs[r]['t_'+startpoint]  # ...at an empirically-derived starting time.
        Tps = [Tp]   # Get ready to receive a temperature history over time...
        ts = [t_start]  # ...and the times associated with each temperature snapshot.
        b = runs[r]['dBdt'] * t_start + runs[r]['b_ints']
        #b = beta_budgets[r].loc[calc.round_up(t_start)]  # Find scaling exponent for heat loss at that starting time.
        H_start = mantle_Ht_max[r].loc[calc.round_up(t_start)]  # Find amount of heat production at starting time, in TW.
        Hts = mantle_Ht_max[r].loc[calc.round_down(t_start):calc.round_up(runs[r]['t_'+endpoint]):direction] # Get span within which to run model.
        
        now = [9.81**b, 2900.0**(3*b), 5.0**(-b), rho_fT(Tp)**(2*b), (alpha_fT(Tp)*Cp_fT(Tp))**b,
               (Tp-dT_fTp-300.0)**(b+1), np.exp(runs[r]['Ea_list']/(calc.R_idealgas*(Tp+dT_fTp)))**(-1*b)]
        ref = [9.81**PT_b, 2900.0**(3*PT_b), 5.0**(-1*PT_b), runs[r]['rho_ref']**(2*PT_b), (runs[r]['alpha_ref']*runs[r]['Cp_ref'])**PT_b,
               runs[r]['deltaT_ref']**(PT_b+1), np.exp(runs[r]['Ea_list']/(calc.R_idealgas*(runs[r]['Tp_present']+dT_fTp)))**(-1*PT_b)]
        refstate = np.prod(ref)
        a = copy.deepcopy(refstate)
        Q_start = runs[r]['Qm_list'] * (np.prod(now)/refstate)  # Estimate present heat loss as multiple of reference state.
        Qt = copy.deepcopy(Q_start)
        Qts = [Qt]
        dT = (direction * runs[r]['starttdiff'] *  # Get temp diff between reference time and pre-calculated grid of times.
                   calc.seconds*1.0e12*(Q_start-H_start)/(Cp_fT(Tp)*calc.M_mant+calc.Cpcore))/timestep
        for t in Hts.index:
            #if Tp>2200.0:
            #    break
            # b = beta_budgets[r].loc[t]
            b= runs[r]['dBdt'] * t + runs[r]['b_ints']
            Tp = Tp + timestep*dT
            Tps.append(Tp)
            ts.append(t)
            Qts.append(Qt)
            refstate = np.prod(copy.deepcopy(now))
            refQ=copy.deepcopy(Qt)
            try:
                now = [9.81**b, 2900.0**(3*b), 5.0**(-b), rho_fT(Tp)**(2*b), (alpha_fT(Tp)*Cp_fT(Tp))**b, 
                       (Tp-dT_fTp-300.0)**(b+1), np.exp(runs[r]['Ea_list']/(calc.R_idealgas*(Tp+dT_fTp)))**(-1*b)]
                #Qt = runs[r]['Qm_list'] * (np.prod(now)/a) #refstate)
                Qt = refQ * (np.prod(now)/refstate)
                dT = direction * (calc.seconds*1.0e12*(Qt-Hts[t])/(Cp_fT(Tp)*calc.M_mant+calc.Cpcore))
            except:
                break
                #now = [9.81**b, 2900**(3*b), 5.0**(-b), runs[r]['rho_ref']**(2*b), (runs[r]['alpha_ref']**runs[r]['Cp_ref'])**b,
                       #(Tp-dT_fTp-300.0)**(b+1), np.exp(runs[r]['Ea_list']/(calc.R_idealgas*(Tp+dT_fTp)))**(-1*b)]
                #Qt = runs[r]['Qm_list'] * (np.prod(now)/a) #refstate)
                #Qt = refQ * (np.prod(now)/refstate)
                #dT = direction * (calc.seconds*1.0e12*(Qt-Hts[t])/(runs[r]['Cp_ref']*calc.M_mant+calc.Cpcore))
        Tlist[r]=Tp+dT*runs[r]['endtdiff']
        t = t+direction * runs[r]['endtdiff'] #+(calc.round_up(runs[r]['t_'+endpoint]) - runs[r]['t_'+endpoint])
        Tps.append(Tlist[r])
        ts.append(t) 
        Qts.append(refQ) #Qt)
        outputs[str(r)+'_'+curve] = pd.Series(data=Tps[1:], index=ts[1:])
        outputs2[str(r)+'_'+curve] = pd.Series(data=Qts[1:], index=ts[1:])
        #plt.plot(ts, Tps, c=assign_colors(my_cm[curve]), alpha=best_alpha)
    cases['Tend'] = Tlist
    cases.to_csv('OUTPUT/'+vary+curve+'.csv')
cases.describe()

In [ ]:
outputs.plot(legend=False, c='w', alpha=best_alpha)
calc.superimpose_Archean_temperatures(cases)
plt.ylim(1750, 2100)

In [ ]:
#cases.plot.scatter('Tend', 'Tp_midarc', rasterized=True)
plt.plot(calc.CI_cols(cases['Tend']))
plt.plot(calc.CI_cols(cases['Tp_midarc']))

In [ ]:
Qsum = calc.CI_rows(outputs2)
mantle_Ht_max.plot(legend=False)
for i in range(len(Qsum.columns)):
    ialpha = 1- np.abs(i-2)/3 # 1/np.abs(np.log(np.abs(i-3)))
    plt.plot(Qsum.index, Qsum[Qsum.columns[i]], rasterized=True, c='orange', alpha=ialpha, lw=3*ialpha)

In [ ]:
#plt.plot(GrowthCurves, rasterized=True)
#plt.plot(beta_budgets[9], c='w')
plt.axvline(x=np.median(cases['t_latearc']), c='r')
plt.axvline(x=np.median(cases['t_midarc']), c='r')
Qsum = calc.CI_rows(outputs2)
for i in range(len(Qsum.columns)):
    ialpha = 1- np.abs(i-2)/3 # 1/np.abs(np.log(np.abs(i-3)))
    plt.plot(Qsum.index, Qsum[Qsum.columns[i]], rasterized=True, c='orange', alpha=ialpha, lw=3*ialpha)
plt.xlim(Qsum.index[0], Qsum.index[-1])
plt.show()

In [ ]:
plt.plot(Hts.index, runs[r]['dBdt'] * Hts.index + runs[r]['b_ints'])

In [ ]:
assign_colors = cm.get_cmap('rainbow', len(calc.curvenames))
my_cm = dict(zip(calc.curvenames, np.linspace(0, 1, len(calc.curvenames))))
plt.figure(figsize=(9,6))
outputs.plot(c='r', alpha=best_alpha, legend=False, rasterized=True)
calc.superimpose_Archean_temperatures(calc.case_defs(nruns=nruns, constants=scenarios['Everything']))
plt.ylim(1700)
plt.show()
#plt.xlim(2.5, 3.9)

In [ ]:
plt.figure(figsize=(9,6))

for i in range(len(curves)):
    temp = outputs[outputs.columns[0+i*nruns:(i+1)*nruns]]
    plt.plot(temp.index, temp, c=assign_colors(my_cm[curves[i]]), rasterized=True, alpha=best_alpha)
    calc.superimpose_Archean_temperatures(cases)
    plt.xlim(2.5,3.9)
    #plt.title(curves[i])

In [ ]:
alldfs = [df for df in [var for var in dir() if isinstance(eval(var), pd.core.frame.DataFrame)] if (df[0]!='_') and (len(i)>2)]
for i in alldfs:
    (globals()[i]).to_csv('OUTPUT/test_'+i+'.csv')